In [1]:
import pyomo.environ as pe
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

To run pyomo with the current SingleMarketSolver implementation you'll need GLPK package:

Download solver for pyomo from here: https://sourceforge.net/projects/winglpk/

Extract it to some folder and specify path to ./glpsol executable for pyomo solver. (as in SingleMarketSolver - pe.SolverFactory('glpk', executable=r"C:\glpk\w64\glpsol.exe"))

In [ ]:
class SingleMarketSolver:

    def __init__(self, **kwargs):
        '''
            Call a method and instantiate any external variables, like:
                state of charge (SoC), 
                state of charge boundaries,
                etc.
        '''
        data = kwargs.get('data', None)
        self.output = []
        self.model = None
        self.X_MTU_optimizer_for_profit(
            data=data,
        )

    def X_MTU_optimizer_for_profit(self, data, **kwargs):
        """
            Implement this method to optimize
        """
        
        # Build the model
        model = pe.ConcreteModel()
        model.T = pe.RangeSet(0, len(data.index) - 1)
        # ...

        # Parameters
        model.ca = pe.Param(model.T, initialize=data['downwards_activation_column'].to_dict())
        # ...

        # Decision variables
        model.charge = pe.Var(model.T, within=pe.Binary)
        # ...

        # Constraints
        model._example_charge_act_con = pe.Constraint(
            model.T, rule=self._example_charge_act_con
        )
        # ...

        # Solve
        solver = pe.SolverFactory('glpk', executable=r"C:\glpk\w64\glpsol.exe")
        solver.options['mipgap'] = 0.01  # Set a 1% relative gap tolerance
        results = solver.solve(model, tee=True)
        print(results.solver.status, results.solver.termination_condition)
        
        # Collect the results
        res = []
        for t in model.T:
            rec = {
                'datetime': data.index[t], # output timestamp, not int here
                'sys_charge_activ': pe.value(model.ca[t]), # output parameters used
                'charge': pe.value(model.charge[t]), # output variables used
            }
            res.append(rec)
        self.output = res
        self.model = model
        return res

    # ---------------------
    # Constraints and objective
    # ---------------------

    def _maximise_profit(self, m):
        '''
            This is an objective function, pyomo passes model object to it (m)
            Implement it for max profit or any other objective
        '''

    def _example_charge_act_con(self, m, t):
        '''
            This is an example constraint, pyomo passes model object (m) and model timestep index (t)
        '''
        # Allow charging only if the system has a down activation.
        return m.charge[t] <= m.ca[t]
        


obj = SingleMarketSolver()
df_results = pd.DataFrame(obj.output)